## Análise de dados das inscrições para o concurso público da SEMED da Prefeitura de Maricá (2024)

### Este projeto visa coletar, tratar e analisar os dados da Lista Geral Preliminar das inscrições confirmadas para o concurso público da SEMED da Prefeitura de Maricá
### Url do arquivo: https://portal.coseac.uff.br/wp-content/uploads/sites/596/2024/04/Concurso-PMM-SEMED20241-ListaGeralPreliminardasinscricoesconfirmadas.pdf
### Data de acesso: 02 de maio de 2024

## Análise inicial do arquivo pdf
### Número de páginas: 1994
### Característiras: Todas as páginas apresentam-se padronizadas com a seguite estrutura:
### - Cabeçalho;
### - Conteúdo: 
Tabela com dados:
    Colunas: Inscrição, Requerimento, Nome do Candidato, Data de Nascimento, Cargo, Tipo de Vaga.
    Linhas: cada linha contém as informações sobre uma inscrição;
### - Rodapé com legendas.

## Bibliotecas Python utilizadas:
### - pdfplumber: para a leitura do arquivo
### - pandas: para a limpeza e tratamento dos dados

In [2]:
import pandas as pd
import pdfplumber

### Código para auxílio na análise exploratória.
### Verificando se os cabeçalhos de todas as páginas são iguais.

In [ ]:
path_to_file = "Concurso-PMM-SEMED20241-ListaGeralPreliminardasinscricoesconfirmadas.pdf"

with pdfplumber.open(path_to_file) as pdf:
    page_0_header = [pdf.pages[0].extract_text().split("\n")[i] for i in range(8) if i != 2]
    
    for i in range(0, 1994):
        page = pdf.pages[i]
        text = page.extract_text()
        page_x_header = [text.split("\n")[i] for i in range(8) if i != 2]
        
        if page_0_header != page_x_header:
            page_number = text.split("\n")[2]
            advise = f'There is diferent header on the page {page_number}'
            print(advise)
            break
    
    print("All headers are the same!")

### Criando e preenchendo uma lista que será usada na geração do dataframe inicial.
### Colunas selecionadas: Cargo pretendido e Tipo de vaga.

In [7]:
path_to_file = "Concurso-PMM-SEMED20241-ListaGeralPreliminardasinscricoesconfirmadas.pdf"
all_lines_to_dataframe = [["Cargo", "TipoVaga"]]

In [8]:
def find_cargo_column(unformated_line):
    docente_position = 0
    for j in range(len(unformated_line)):
        if "Docente" in unformated_line[j]:
            docente_position = j
    return(docente_position)

In [15]:
with pdfplumber.open(path_to_file) as pdf:
       
    for i in range(1500, 1994):
        page = pdf.pages[i]
        text = page.extract_text()
        
        if "Inscritos: 69770" in text: #Verifica se é a última página
            for i in range(8, 23):
                unformated_line = text.split('\n')[i].split(' ')
                all_lines_to_dataframe.append([" ".join(unformated_line[find_cargo_column(unformated_line):-1]), unformated_line[-1]])
        else:
            counter = 0
            limit_line_number = 0
            for line in text.split('\n'):
                counter = counter + 1
                if "Tipo de Vaga" in line:
                    limit_line_number = counter
           
            for i in range(8, limit_line_number-1):
                unformated_line = text.split('\n')[i].split(' ')
                all_lines_to_dataframe.append([" ".join(unformated_line[find_cargo_column(unformated_line):-1]), unformated_line[-1]])

## Gerando o dataframe inicial

In [ ]:
df = pd.DataFrame(all_lines_to_dataframe)
df.columns = df.iloc[0]
df = df[1:]
df

## Criação de uma arquivo .csv com os dados sobre Cargo e Tipo de Vaga

In [17]:
df.to_csv('Marica_Cargo_TipoVaga.csv')

## Análise prévia do arquivo csv gerado a partir dos dados originais

In [ ]:
df_from_csv = pd.read_csv("Marica_Cargo_TipoVaga.csv")
df_from_csv = df_from_csv[["Cargo", "TipoVaga"]]
df_from_csv

In [ ]:
df_from_csv.info()

## Agrupando os dados por Cargo e gerando um json

In [ ]:
grouped_data_1 = df_from_csv.groupby(["Cargo"]).size().reset_index(name="Total")
grouped_data_1

In [ ]:
print(grouped_data_1.to_json(orient='records', force_ascii=False))